In [101]:
# import the necessary packages
import os
import cv2
import skimage
import pandas as pd
import numpy as np
import imutils
from imutils.perspective import four_point_transform
from imutils import contours
from skimage import io
from PIL import Image

In [102]:
# define some functions
def getListOfFiles(dirName):
    # create a list of file and sub directories names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [103]:
# set folder name (including path from this jupyter notebook)
folder = "photos"

# set paper size
paper_size = 0.06032246 # use units of m^2

# build list of image files
file_list = getListOfFiles(folder)
if 'photos/.DS_Store' in file_list:
    file_list.remove('photos/.DS_Store') # remove this, if it exists

In [104]:
# set expected bubble number
bubbleNumber = 20

# define each category, on each line in "scamtron"
vineID = {0: "A", 1: "B", 2: "C", 3: "D", 4: "Other"}
zCat = {0: "Low", 1: "Middle", 2: "High", 3: "NA3", 4: "NA4"}
compassCat = {0: "N", 1: "E", 2: "S", 3: "W", 4: "NA4"}
line4 = {0: "NA0", 1: "NA1", 2: "NA2", 3: "NA3", 4: "NA4"}

In [105]:
# reset some variables
z = 0
df = pd.DataFrame()
sus_list = []

# loop over all images in the list of image files
while z < len(file_list):
    # set image name
    imageName = file_list[z]

    # load the image
    image = cv2.imread(imageName)

    # compute ratio of old height to the new height, clone it and resize it
    ratio = image.shape[0] / 1000.0 # save in case we want to rescale to full size later
    orig = image.copy()
    image = imutils.resize(image, height = 1000)

    # convert resized image to grayscale, blur it slightly, then find edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 75, 200)
    
    # apply Otsu's thresholding method to create a binarized version, too
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    # find contours in the edge map, keeping only the largest ones
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

    # loop over the contours
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True) # approximate the contour
        if len(approx) == 4: # if approx contour has four points, then assume that we found paper
            screenCnt = approx
            break
            
    # apply the four point transform to obtain a top-down view of the paper area only         
    warped = four_point_transform(thresh, screenCnt.reshape(4, 2)) 
    warped_color = four_point_transform(image, screenCnt.reshape(4, 2))
    
    # find contours in the thresholded and warped image, then initialize the list of contours that correspond to bubbles
    cnts = cv2.findContours(warped.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    questionCnts = []
    # loop over the contours
    for c in cnts:
        # compute the bounding box of the contour, then use the bounding box to derive the aspect ratio
        (x, y, w, h) = cv2.boundingRect(c)
        ar = w / float(h)
        # in order to label the contour as a question, region should be sufficiently wide, sufficiently tall, and 
        # have an aspect ratio approximately equal to 1
        if warped.shape[1] * 0.076 >= w >= warped.shape[1] * 0.0126 and warped.shape[0] * 0.0556 >= h >= warped.shape[0] * 0.013 and ar >= 0.85 and ar <= 1.15:
            questionCnts.append(c)
        color = (255, 0, 0) # red
        # draw the outline of the bubble on the test
        cv2.drawContours(warped_color, cnts, -1, color, 3)
    
#     # test
#     io.imshow(warped_color)
#     io.show()
#     print(len(questionCnts))
    
    if len(questionCnts) == bubbleNumber: # only record data when all bubbles are found
        # sort the question contours top-to-bottom, then initialize the total number of correct answers
        questionCnts = contours.sort_contours(questionCnts,method="top-to-bottom")[0]

        correct = 0
        set_vineID = 0
        set_zCat = 0
        set_compassCat = 0
        set_line4 = 0

        # each question has 5 possible answers, to loop over the question in batches of 5
        for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
            # sort the contours for the current question from left to right, then initialize the index of the bubbled answer
            cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
            bubbled = None
            # loop over the sorted contours
            for (j, c) in enumerate(cnts):
                # construct a mask that reveals only the current"bubble" for the question
                mask = np.zeros(warped.shape, dtype="uint8")
                cv2.drawContours(mask, [c], -1, 255, -1)
                # apply the mask to the thresholded image, then count the number of non-zero pixels in the bubble area
                mask = cv2.bitwise_and(warped, warped, mask=mask)
                total = cv2.countNonZero(mask)
                # if the current total has a larger number of total non-zero pixels, then we are examining the currently bubbled-in answer
                if bubbled is None or total > bubbled[0]:
                    bubbled = (total, j)
            # vine ID readout option
            if q == 0:
                set_vineID = vineID[bubbled[1]]
            elif q == 1:
                set_zCat = zCat[bubbled[1]]
            elif q == 2:
                set_compassCat = compassCat[bubbled[1]]
            elif q == 3:
                set_line4 = line4[bubbled[1]] 

        # masking everything but the leaf
        warped2 = warped.copy()
        start = warped2.shape[0]*0.7
        end = warped2.shape[1]*0.4
        warped2[int(start):,:int(end)] = 0  

        # extract leaf area
        n_white = cv2.countNonZero(warped2)
        height, width = warped2.shape
        n_total = height * width
        leaf_area = (n_white / n_total) * paper_size

        # extract data from image metadata
        creation_time = Image.open(imageName)._getexif()[36867]

        if z == 0:
            
            # initialize list of lists
            data = [[creation_time[:10], set_vineID, set_zCat, set_compassCat, leaf_area]]

            # Create the pandas DataFrame from list of lists
            df = pd.DataFrame(data, columns=['Date', 'Plant', 'zCat', 'compassCat', 'Leaf Area'])

        else:
            # append onto existing results
            df2 = {'Date': creation_time[:10], 'Plant': set_vineID, 'zCat': set_zCat, 'compassCat': set_compassCat, 'Leaf Area': leaf_area}
            df = df.append(df2, ignore_index = True)

        z += 1

    else:  # otherwise, add filename to list for supervised (human) analysis
        index = len(sus_list)
        sus_list.insert(index, imageName)
        
        z += 1

display(df)
print(sus_list)

,Date,Leaf Area,Plant,compassCat,zCat
0,2022:08:05,0.005179,A,S,Middle
1,2022:08:05,0.001367,A,S,Middle
2,2022:08:05,0.018287,A,S,Middle
3,2022:08:03,0.012478,D,N,Middle
4,2022:08:05,0.003024,A,S,Middle


['photos/IMG_0097.jpeg', 'photos/IMG_0100.jpeg', 'photos/IMG_0099.jpeg']


In [106]:
print(file_list)

['photos/IMG_0097.jpeg', 'photos/IMG_0096.jpeg', 'photos/IMG_0100.jpeg', 'photos/IMG_0088.jpeg', 'photos/IMG_0099.jpeg', 'photos/IMG_0095.jpeg', 'photos/IMG_0077.jpeg', 'photos/IMG_0098.jpeg']
